https://developers.google.com/gmail/api/quickstart/python

note: the script below must be run as a .py file in order to launch your browser and authenticate

In [1]:
!hostname

b9776dd61592


In [2]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /opt/conda/lib/python3.6/site-packages
Requirement already up-to-date: google-auth-httplib2>=0.0.3 in /opt/conda/lib/python3.6/site-packages (from google-api-python-client)
Requirement already up-to-date: six<2dev,>=1.6.1 in /opt/conda/lib/python3.6/site-packages (from google-api-python-client)
Requirement already up-to-date: httplib2<1dev,>=0.9.2 in /opt/conda/lib/python3.6/site-packages (from google-api-python-client)
Requirement already up-to-date: uritemplate<4dev,>=3.0.0 in /opt/conda/lib/python3.6/site-packages (from google-api-python-client)
Requirement already up-to-date: google-auth>=1.4.1 in /opt/conda/lib/python3.6/site-packages (from google-api-python-client)
Requirement already up-to-date: pyasn1-modules>=0.2.1 in /opt/conda/lib/python3.6/site-packages (from google-auth>=1.4.1->google-api-python-client)
Requirement already up-to-date: cachetools>=2.0.0 in /opt/conda/lib/python3.6/site-packages (from google-auth>=1

In [3]:
!pip install --upgrade oauth2client

Requirement already up-to-date: oauth2client in /opt/conda/lib/python3.6/site-packages
Requirement already up-to-date: httplib2>=0.9.1 in /opt/conda/lib/python3.6/site-packages (from oauth2client)
Requirement already up-to-date: six>=1.6.1 in /opt/conda/lib/python3.6/site-packages (from oauth2client)
Requirement already up-to-date: pyasn1>=0.1.7 in /opt/conda/lib/python3.6/site-packages (from oauth2client)
Requirement already up-to-date: rsa>=3.1.4 in /opt/conda/lib/python3.6/site-packages (from oauth2client)
Requirement already up-to-date: pyasn1-modules>=0.0.5 in /opt/conda/lib/python3.6/site-packages (from oauth2client)
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

https://developers.google.com/gmail/api/guides/sending

message text needs to be base64 encoded

In [5]:
import base64
base64.b64encode(bytes('your string', 'utf-8'))

b'eW91ciBzdHJpbmc='

In [6]:
#https://docs.python.org/3/library/email.mime.html
from email.mime.text import MIMEText
from email import encoders
    
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    
    # https://docs.python.org/3/library/base64.html
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}
    # https://stackoverflow.com/questions/606191/convert-bytes-to-a-string
    
    #return {'raw': base64.urlsafe_b64encode(bytes("None",'utf-8'))}
    
    # https://docs.python.org/3.4/library/email-examples.html
    #return {'raw': encoders.encode_base64(message.as_string())} 

In [7]:
msg=create_message("benpayne@umbc.edu", 
                   "ben.is.located@gmail.com", 
                   "a subjct", 
                   "msg text from Python")
msg

{'raw': 'Q29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1cy1hc2NpaSIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogN2JpdAp0bzogYmVuLmlzLmxvY2F0ZWRAZ21haWwuY29tCmZyb206IGJlbnBheW5lQHVtYmMuZWR1CnN1YmplY3Q6IGEgc3ViamN0Cgptc2cgdGV4dCBmcm9tIFB5dGhvbg=='}

In [8]:
from googleapiclient import errors

def send_message(service, user_id, message):
    """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print ('Message Id: %s' % message['id'])
        return message
    # https://stackoverflow.com/questions/42541857/error-using-gmail-api-tuto-using-python-3-except-errors-httperror-error
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)

In [9]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

In [10]:
send_message(service, "me", msg)

Message Id: 166c28947f5cfea4


{'id': '166c28947f5cfea4',
 'threadId': '166c28947f5cfea4',
 'labelIds': ['UNREAD', 'SENT', 'INBOX']}

The following also works